### Ce jupiter traitera toutes les insertions dans mongodb

#### Intallation et chargement des bibliothèques

In [ ]:
python -m pip install pymongo 

In [2]:
import pandas as pd
from pymongo import MongoClient

#### Chargement des fichiers

In [6]:
immo_brut = pd.read_csv('./data/france_total_real_estate_sales_2022.csv', dtype=str)
print(immo_brut)

        Date mutation Nature mutation Valeur fonciere No voie Code voie  \
0          03/01/2022           Vente        55000,00    13.0      2280   
1          03/01/2022           Vente       143000,00     NaN      B010   
2          03/01/2022           Vente       143000,00     NaN      B010   
3          03/01/2022           Vente       143000,00    98.0      0055   
4          04/01/2022           Vente          300,00     NaN      B031   
...               ...             ...             ...     ...       ...   
3803880    30/12/2022           Vente      3500000,00    19.0      4879   
3803881    19/12/2022           Vente       525000,00     3.0      3842   
3803882    19/12/2022           Vente       525000,00     3.0      3842   
3803883    30/12/2022           Vente        15500,00    69.0      0679   
3803884    21/11/2022           Vente        30000,00    48.0      8358   

        Code postal            Commune Code departement Code commune Section  \
0            1000.0